In [ ]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.IO;

In [ ]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("StackOverflow");
var postContainer = db.GetContainer("Posts");

In [ ]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
}

A User adds a new question which triggers an INSERT operation.

In [42]:
var newpost = new Post
{
    id = Guid.NewGuid().ToString(),
    PostId = -10,
    PostBody = "Can anyone help me with creating a new record in a Cosmos DB container using C#?",
    Title = "How to create a new record in a Cosmos DB container?",
    ViewCount = 0,
    AnswerCount = 0,
    CommentCount = 0,
    FavoriteCount = 0,
    AcceptedAnswerId = 0,
    CreatedOn = DateTime.UtcNow,
    ClosedDate = null,
    OwnerUserId = -10,
    OwnerDisplayName = "Test User",
    PostType = "Question",
    Score = 0,
    Tags = "CosmosDB,Azure,Database"    
};

var newitem = await postContainer.CreateItemAsync(newpost, new PartitionKey(newpost.OwnerUserId));
Console.WriteLine($"Created item in database with id: {newitem.Resource.id}");
Console.WriteLine($"Request charge: {newitem.RequestCharge}");



Created item in database with id: 3db1288d-80d0-4606-bb90-c9b5393face0
Request charge: 11.05


A User finds this question and clicks on it.

In [43]:
var id = newitem.Resource.id;
var userid = newitem.Resource.OwnerUserId;

var postQuery = new QueryDefinition("SELECT * FROM c WHERE c.id = @id AND c.OwnerUserId = @userid")
    .WithParameter("@id", id)
    .WithParameter("@userid", userid);
    // You can change the consistency level of READ here.
    // var iterator = postContainer.GetItemQueryIterator<Post>(postQuery, requestOptions: new QueryRequestOptions { ConsistencyLevel = ConsistencyLevel.Eventual });
var iterator = postContainer.GetItemQueryIterator<Post>(postQuery);
var results = new List<Post>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
    Console.WriteLine($"Request charge: {response.RequestCharge}");
}
results[0].Display();

Request charge: 2.92


id,3db1288d-80d0-4606-bb90-c9b5393face0
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,0
AnswerCount,0
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-09-26 01:22:32Z
ClosedDate,<null>


If you can make id and partitionid available, you can make this operation much more affordable and fast by using READ operation rather than QUERY

In [44]:
var post = await postContainer.ReadItemAsync<Post>(id, new PartitionKey(userid));
Console.WriteLine($"Request charge: {post.RequestCharge}");
post.Resource.Display();

Request charge: 1


id,3db1288d-80d0-4606-bb90-c9b5393face0
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,0
AnswerCount,0
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-09-26 01:22:32Z
ClosedDate,<null>


A User answers the question which triggers an Update Operation.

In [45]:
var query = new QueryDefinition("SELECT * FROM c WHERE c.PostId = @postId")
    .WithParameter("@postId", newpost.PostId);
var iterator = postContainer.GetItemQueryIterator<Post>(query);
var results = new List<Post>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
}
results[0].Display();
results[0].ViewCount += 1;
results[0].AnswerCount += 1;
results[0].Score += 10;
//Figure out the partition key
var updateResponse = await postContainer.ReplaceItemAsync(results[0], results[0].id, new PartitionKey(results[0].OwnerUserId));
updateResponse.Resource.Display();
Console.WriteLine($"Request charge: {updateResponse.RequestCharge}");

id,3db1288d-80d0-4606-bb90-c9b5393face0
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,0
AnswerCount,0
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-09-26 01:22:32Z
ClosedDate,<null>


id,3db1288d-80d0-4606-bb90-c9b5393face0
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,1
AnswerCount,1
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-09-26 01:22:32Z
ClosedDate,<null>


Request charge: 11.43


We can use Partial Update to make this code and request package lighter.
Partial Update request contains only the properties needs to get updated.

In [46]:
var _id = newitem.Resource.id;
var pkey = new PartitionKey(newitem.Resource.OwnerUserId);

var ops = new List<PatchOperation>();
ops.Add(PatchOperation.Increment("/ViewCount", 1));
ops.Add(PatchOperation.Increment("/AnswerCount", 1));
ops.Add(PatchOperation.Increment("/Score", 10));

var result = await postContainer.PatchItemAsync<Post>(
    id: _id, 
    partitionKey: pkey, 
    patchOperations: ops);

Console.WriteLine($"Request charge: {result.RequestCharge}");
result.Resource.Display();


Request charge: 11.8


id,3db1288d-80d0-4606-bb90-c9b5393face0
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,2
AnswerCount,2
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-09-26 01:22:32Z
ClosedDate,<null>


Another way to query data is the STREAM API
Partition Key is required.
There will be no serialization which will make the operation lighter.

In [47]:
var query = new QueryDefinition("SELECT * FROM c WHERE c.PostId = @postId")
    .WithParameter("@postId", newitem.Resource.PostId);
var iterator = postContainer.GetItemQueryStreamIterator(query,null, new QueryRequestOptions { PartitionKey = new PartitionKey(newitem.Resource.PostId) });
JObject result;
while (iterator.HasMoreResults)
{
    using (ResponseMessage response = await iterator.ReadNextAsync())
    {
        using (StreamReader sr = new StreamReader(response.Content))
        {
            string content = await sr.ReadToEndAsync();
            result = JObject.Parse(content);
            

        }
        /*using (JsonTextReader jtr = new JsonTextReader(sr))
        {
            result = JObject.Load(jtr);
            result.Display();            
        }*/
    }
}
result.ToString().Display();

{
  "_rid": "4QYyANtoaJI=",
  "Documents": [
    {
      "id": "3db1288d-80d0-4606-bb90-c9b5393face0",
      "PostId": -10,
      "PostBody": "Can anyone help me with creating a new record in a Cosmos DB container using C#?",
      "Title": "How to create a new record in a Cosmos DB container?",
      "ViewCount": 2,
      "AnswerCount": 2,
      "CommentCount": 0,
      "FavoriteCount": 0,
      "AcceptedAnswerId": 0,
      "CreatedOn": "2025-09-26T01:22:32.7775895Z",
      "ClosedDate": null,
      "OwnerUserId": -10,
      "OwnerDisplayName": "Test User",
      "PostType": "Question",
      "Score": 20,
      "Tags": "CosmosDB,Azure,Database",
      "_rid": "4QYyANtoaJIHAAAAAAAAAA==",
      "_self": "dbs/4QYyAA==/colls/4QYyANtoaJI=/docs/4QYyANtoaJIHAAAAAAAAAA==/",
      "_etag": "\"00000000-0000-0000-2e84-236d341b01dc\"",
      "_attachments": "attachments/",
      "_ts": 1758849797
    }
  ],
  "_count": 1
}

System finds this question a duplicate and deletes the question

In [48]:
var id = results[0].id;
var pkey = userid;
var deleteResponse = postContainer.DeleteItemAsync<Post>(id, new PartitionKey(pkey));
Console.WriteLine($"Delete Operation Cost {deleteResponse.Result.RequestCharge} RU/s");

Delete Operation Cost 11.05 RU/s
